# Imports

In [1]:
import tensorflow as tf

# Dataset
import numpy as np
import pandas as pd

# # Utils
# from datetime import datetime

# Constants

In [2]:
# Model
learning_rate = 5e-2
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.Huber()
metrics = ["mae", "mse"]
epochs = 1000

# Data split
test_split = 0.1                # Float or int
valid_split = 0.1               # Float
train_split = 1 - valid_split   # Float

# Dataset window
steps_size = 15               # minutes
predicts_size = 1             # minute(s)
window_size = steps_size + predicts_size
batch_size = 32
shuffle_buffer_size = 64

# Dataset frame
num_of_features = 6
num_of_labels = 2

# File
dataset_file = './dataset/user_1/user_1_data.csv'
model_1_file = './model/user_1/'
model_2_file = './model/...'

# Data

## Fetching

In [3]:
data = pd.read_csv(dataset_file)

data

,date,time,latitude,longitude
0,3/7/2022,0:00:00,-6.268917,106.780112
1,3/7/2022,0:01:00,-6.268917,106.780112
2,3/7/2022,0:02:00,-6.268917,106.780112
3,3/7/2022,0:03:00,-6.268917,106.780112
4,3/7/2022,0:04:00,-6.268917,106.780112
...,...,...,...,...
20155,3/20/2022,23:55:00,-6.268917,106.779552
20156,3/20/2022,23:56:00,-6.268917,106.779552
20157,3/20/2022,23:57:00,-6.268917,106.779552
20158,3/20/2022,23:58:00,-6.268917,106.779552


## Preprocessing

In [4]:
# Converting date string to datetime
data["date"] = pd.to_datetime(data["date"])
data["day_of_week"] = data["date"].dt.day_of_week
data["month"] = data["date"].dt.month
data["year"] = data["date"].dt.year

# Converting time to cumulative minute
# source: https://stackoverflow.com/questions/17951820/convert-hhmmss-to-minutes-using-python-pandas
# credit: Andy Hayden
data["time"] = data["time"].str.split(':').apply(lambda time: int(time[0]) * 60 + int(time[1]))

# Removing unused column
del data["date"]

# Rearrange column
data = data[["year", "month", "day_of_week", "time", "latitude", "longitude"]]

data

,year,month,day_of_week,time,latitude,longitude
0,2022,3,0,0,-6.268917,106.780112
1,2022,3,0,1,-6.268917,106.780112
2,2022,3,0,2,-6.268917,106.780112
3,2022,3,0,3,-6.268917,106.780112
4,2022,3,0,4,-6.268917,106.780112
...,...,...,...,...,...,...
20155,2022,3,6,1435,-6.268917,106.779552
20156,2022,3,6,1436,-6.268917,106.779552
20157,2022,3,6,1437,-6.268917,106.779552
20158,2022,3,6,1438,-6.268917,106.779552


## Splitting

In [5]:
def split_data(data, train_split, test_split):
    """Split data to train, valid, and test data"""
    # Split train_valid data and test data
    test_len = test_split
    if type(test_split)==float:
        test_len = int(test_len * len(data))
    train_val_data, test_data = data[:-test_len], data[-test_len:]
    
    # Split train data and valid data
    train_len = int(len(train_val_data) * train_split)
    train_data, valid_data = train_val_data[:train_len], train_val_data[train_len:]
    
    return train_data, valid_data, test_data

train_data, valid_data, test_data = split_data(data, train_split, test_split)

print("Dataset Shape")
print(f'Train : {train_data.shape}')
print(f'Valid : {valid_data.shape}')
print(f'Test  : {test_data.shape}')

Dataset Shape
Train : (16329, 6)
Valid : (1815, 6)
Test  : (2016, 6)


## Windowing

In [6]:
def windowed_dataset(data, steps_size, predicts_size, batch_size, shuffle_buffer):
    """Create windowed dataset"""
    # Converting to tfds
    wds = tf.data.Dataset.from_tensor_slices(data)
    
    # Data shifting
    wds = wds.window(steps_size+predicts_size, shift=predicts_size, drop_remainder=True)
    
    # Flatten windows
    wds = wds.flat_map(lambda window : window.batch(steps_size+predicts_size))
    
    # Create window tuples
    wds = wds.map(lambda window: (window[:-predicts_size], window[-predicts_size:, -num_of_labels:]))
    
    # Shuffle windows
    wds = wds.shuffle(shuffle_buffer)
    
    # Batch windows
    wds = wds.batch(batch_size).prefetch(1)
    
    return wds

wds = windowed_dataset(data, steps_size, predicts_size, batch_size, shuffle_buffer_size)
for idx,(x,y) in enumerate(wds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    break

x =  (32, 15, 6)
y =  (32, 1, 2)


In [7]:
train_wds = windowed_dataset(train_data, steps_size, predicts_size, batch_size, shuffle_buffer_size)
valid_wds = windowed_dataset(valid_data, steps_size, predicts_size, batch_size, shuffle_buffer_size)
test_wds = windowed_dataset(test_data, steps_size, predicts_size, batch_size, shuffle_buffer_size)

# Model

## Base Model

In [81]:
def create_model():
    """Create Forecasting Model
    Model used: LSTM
    output should consist of 2 item, latitude and longitude
    """
    tf.keras.backend.clear_session()
    # Generating model
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, activation='sigmoid', input_shape=(steps_size, num_of_features), return_sequences=True),
        tf.keras.layers.LSTM(32, activation='sigmoid', return_sequences=True),
        tf.keras.layers.LSTM(16, activation='sigmoid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation='sigmoid'),
        tf.keras.layers.Dense(8, activation='sigmoid'),
        tf.keras.layers.Dense(num_of_labels, activation='linear')
    ])

    # Compiling model
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=metrics,
    )
    
    return model
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 64)            18176     
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            12416     
                                                                 
 lstm_2 (LSTM)               (None, 16)                3136      
                                                                 
 flatten (Flatten)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 2)                 1

## Training

In [82]:
model.fit(train_wds, epochs=20, validation_data=valid_wds)

Epoch 1/20
510/510 [==============================] - 11s 16ms/step - loss: 2.4387 - mae: 2.5161 - mse: 171.5694 - val_loss: 7.3463e-04 - val_mae: 0.0320 - val_mse: 0.0015
Epoch 2/20
510/510 [==============================] - 8s 15ms/step - loss: 1.7369e-04 - mae: 0.0128 - mse: 3.4742e-04 - val_loss: 6.3544e-04 - val_mae: 0.0276 - val_mse: 0.0013
Epoch 3/20
510/510 [==============================] - 8s 16ms/step - loss: 1.4363e-04 - mae: 0.0111 - mse: 2.8710e-04 - val_loss: 6.1483e-04 - val_mae: 0.0267 - val_mse: 0.0012
Epoch 4/20
510/510 [==============================] - 8s 15ms/step - loss: 1.3278e-04 - mae: 0.0106 - mse: 2.6558e-04 - val_loss: 6.2296e-04 - val_mae: 0.0265 - val_mse: 0.0012
Epoch 5/20
510/510 [==============================] - 9s 17ms/step - loss: 1.2567e-04 - mae: 0.0100 - mse: 2.5122e-04 - val_loss: 6.7220e-04 - val_mae: 0.0287 - val_mse: 0.0013
Epoch 6/20
510/510 [==============================] - 9s 17ms/step - loss: 1.3820e-04 - mae: 0.0105 - mse: 2.7633e-04 - 

## Evaluating Model

In [83]:
model.evaluate(test_wds)

63/63 [==============================] - 0s 6ms/step - loss: 9.3190e-04 - mae: 0.0359 - mse: 0.0019


[0.0009318969678133726, 0.035878490656614304, 0.0018645442323759198]

## Using Model

### Converting Data

In [84]:
def convert_data(data):
    """Convert data to model input"""
    # Take last "steps size" data from copy data
    cdata = data.copy()[-steps_size:]
    if len(cdata) != steps_size:
        # Not enough data to do prediction
        return None
    # Add empty row in the bottom
    cdata.loc[cdata.shape[0]] = np.zeros(num_of_features)
    
    return windowed_dataset(cdata, steps_size, predicts_size, batch_size, shuffle_buffer_size)

predict_data = convert_data(data)
predict_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>

### Predicting Data

In [97]:
def predict(model, data):
    """Predict data using model and data"""
    steps_data = convert_data(data)
    if not steps_data:
        # If converting failed (not enough data) return none
        return None
    # Else return prediction
    return  model.predict(
        steps_data
    )


print("\n\n========================================")
print("Predict")
print("========================================\n")
print(predict(model, data[:6763]))

print("\n\n========================================")
print("Label")
print("========================================\n")
print([[data.loc[6763]["latitude"], data.loc[6763]["longitude"]]])



Predict

1/1 [==============================] - 0s 39ms/step
[[ -6.2238636 106.81572  ]]


Label

[[-6.229087622, 106.7979363]]


# References

- [Sequences, Time Series and Prediction by DeepLearning.AI](https://www.coursera.org/learn/tensorflow-sequences-time-series-and-prediction)
- [Multi-Variate Time Series Forecasting Tensorflow by Nicholas Jhana](https://www.kaggle.com/code/nicholasjhana/multi-variate-time-series-forecasting-tensorflow/notebook#Visualizing-Predictions)